# Recommendation

Content Based --> Attributes of user or item

Colloborative filtering --> Memory based {User based, Item Based} , Model Based {Nearest Neighbour , Cosine, Euclidean distance, Manhattan distance, SVD}

Hybrid --> Combination of Content & Colloborative 

# Item, User based collaborative filtering python

In [ ]:
#importing neccesary packages
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Step 1 - Data Preparation

In [ ]:
#Reading movielens data from 
#https://grouplens.org/datasets/movielens/
my_df = pd.read_csv('D:/data/u.data', sep='\t', names=['user_id','item_id','rating','timestamp'])

In [ ]:
#Checking Data
my_df.head()

In [ ]:
help(pd.read_csv)

In [ ]:
#Importing movie title and joining with main data
movie_titles = pd.read_csv('Movie_Titles.csv',encoding= 'unicode_escape')
my_df = pd.merge(my_df, movie_titles, on='item_id')
my_df.head()

In [ ]:
#Statitical features of the movie
my_df.describe()

In [ ]:
my_df.shape

In [ ]:
#creating mean ratings data
ratings = pd.DataFrame(my_df.groupby('title')['rating'].mean())
ratings.head()

In [ ]:
#creating number of ratings data
ratings['number_of_ratings'] = my_df.groupby('title')['rating'].count()
ratings.head()

In [ ]:
#Plotting the jointplot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.jointplot(x='rating', y='number_of_ratings', data=ratings)


In [ ]:
my_df

In [ ]:
my_df['item_id'].nunique()

In [ ]:
my_df['title'].nunique()

In [ ]:
my_df['user_id'].nunique()

# Creating User - Item interaction matrix

In [63]:
#creating the user-item interaction matrix
movie_matrix_UII = my_df.pivot_table(index='user_id', columns='title', values='rating')
movie_matrix_UII.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0


In [ ]:
movie_matrix_UII.shape

In [ ]:
#Most rated movies
ratings.sort_values('number_of_ratings', ascending=False).head(10)

In [ ]:
#Fetching ratings for Fargo
Scream_user_rating = movie_matrix_UII['Scream (1996)']

In [ ]:
Scream_user_rating.mean()

In [ ]:
#Finding the correlation with different movies
similar_to_Scream=movie_matrix_UII.corrwith(Scream_user_rating)

In [ ]:
#Observing the result
similar_to_Scream.head()

In [ ]:
ratings

# creating a threshold for minimum number of ratings

In [58]:
#creating dataframe to bring in #of ratings
corr_Scream = pd.DataFrame(similar_to_Scream, columns=['Correlation'])
corr_Scream.dropna(inplace=True)
corr_Scream.head()

,Correlation
title,
'Til There Was You (1997),0.454859
1-900 (1994),-0.944911
101 Dalmatians (1996),0.124059
12 Angry Men (1957),-0.025659
187 (1997),0.382836


In [61]:
#Bringing in ratings
corr_Scream = corr_Scream.join(ratings['number_of_ratings'])

corr_Scream.head()

,Correlation,number_of_ratings
title,,
'Til There Was You (1997),0.454859,9
1-900 (1994),-0.944911,5
101 Dalmatians (1996),0.124059,109
12 Angry Men (1957),-0.025659,125
187 (1997),0.382836,41


In [62]:
corr_Scream[corr_Scream['number_of_ratings'] > 100].sort_values(by='Correlation', ascending=False).head(10)

,Correlation,number_of_ratings
title,,
Scream (1996),1.000000,478
Scream 2 (1997),0.706028,106
Seven (Se7en) (1995),0.435188,236
Starship Troopers (1997),0.419322,211
"Nightmare on Elm Street, A (1984)",0.410796,111
Cape Fear (1991),0.397245,171
Interview with the Vampire (1994),0.386182,137
Natural Born Killers (1994),0.383332,128
Young Guns (1988),0.381230,101
